In [10]:
import requests
import json

In [9]:
# a request to localhost:8080

url = "http://localhost:8080"
# url = "https://mypersonaljscloudivnad9dy-readnft.functions.fnc.fr-par.scw.cloud"

my_prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Where is Chile?"}
 ]
# my_prompt = "Where is Chile?"  # Example prompt, can be replaced with any string


# Sign the message

Now that we create messages and do not directly write them to the blockchain, we need to sign them before sending them to the server.

In [3]:
import time
import json
import hashlib
import random
import requests
from web3 import Web3
from eth_account.messages import encode_defunct
from urllib.parse import urlparse

from dotenv import load_dotenv
import os

from typing import Optional, Dict, Any

In [7]:
load_dotenv()
WALLET_PRIVATE_KEY = os.getenv("TEST_WALLET_PRIVATE_KEY")

In [ ]:
class EthApiAuth:
    def __init__(self, api_base_url, private_key):
        self.api_base_url = api_base_url
        self.w3 = Web3()
        self.account = self.w3.eth.account.from_key(private_key)

    def create_auth_payload(self, endpoint: str, data, method: str = 'POST') -> Dict[str, Any]:
        """Create authentication payload with signature"""
        if not self.account:
            raise ValueError("No account set. Provide private key first.")

        timestamp = int(time.time() * 1000)  # milliseconds
        nonce = random.randint(100000, 999999)

        # Create standardized message components
        message_parts = [
            f"{self.api_base_url} API Request",
            f"Address: {self.account.address}",
            f"Endpoint: {endpoint}",
            f"Method: {method}",
            f"Timestamp: {timestamp}",
            f"Nonce: {nonce}"
        ]

        # Include data hash if present
        data_hash = None
        if not data:
            raise ValueError("Data cannot be None or empty.")
        # Sort keys for consistent hashing
        data_string = json.dumps(data, sort_keys=True, separators=(',', ':'))
        data_hash = hashlib.sha256(data_string.encode()).hexdigest()
        message_parts.append(f"Data Hash: {data_hash}")

        message = '\n'.join(message_parts)

        # Sign the message
        signature = self.sign_message(message)

        return {
            'auth': {
                'address': self.account.address,
                'signature': signature,
                'timestamp': timestamp,
                'nonce': nonce,
                'message': message,
                'dataHash': data_hash
            },
            'data': data
        }

    def sign_message(self, message):
        """Sign message with private key"""
        if not self.account:
            raise ValueError("No account set. Provide private key first.")

        # Encode message for signing
        message_encoded = encode_defunct(text=message)

        # Sign the message
        signed_message = self.account.sign_message(message_encoded)

        return signed_message.signature.hex()

    def make_authenticated_request(self, endpoint, method="POST", data=None):
        """Make authenticated API request"""
        # Create auth message
        auth_data = self.create_auth_payload(endpoint, data, method)

        # Make request
        url = f"{self.api_base_url}{endpoint}"

        if method.upper() == "POST":
            response = requests.post(url, json=auth_data)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        print(response.status_code, response.text)

In [18]:
url = "http://localhost:8080"

auth = EthApiAuth(url, WALLET_PRIVATE_KEY)
prompt_data = {"prompt": my_prompt}

# Make authenticated POST request
user_data = auth.make_authenticated_request("/", method="POST", data=prompt_data)
print("User data:", user_data)


200 I am a placeholder for the LLM response


JSONDecodeError: Expecting value: line 1 column 1 (char 0)